## Inputs

In [202]:
table_name= "S_program_oe_rate"
table_name= table_name.lower()

In [203]:
text= """[Enrollment Id] = [S_staged_next_enrollment.Enrollment Id]
[Student Id] = [S_staged_next_enrollment.Student Id]
[Client Id] = [S_staged_next_enrollment.Client Id]
[Program OE Id] = [S_staged_next_enrollment.Current Program OE Id]
[Next Enrollment Id] = [S_staged_next_enrollment.Next Enrollment Id]
[Next Program OE Id] = [S_staged_next_enrollment.Next Program OE Id]
[Previous Enrollment Id] = [S_staged_previous_enrollment.Previous Enrollment Id]
[Previous Program OE Id] = [S_staged_previous_enrollment.Previous Program OE Id]"""

## Process

In [204]:
import re
sp_text= list(map(lambda x: x.split('='), text.split('\n')))

In [205]:
#Variable label names

def clean_label(word):
    word= word.lower() \
        .strip() \
        .replace(' ', '_') \
        .replace('[', '') \
        .replace(']', '')
    return word

def is_nvarchar(word):
    if 'id' in word: return '[NVARCHAR] (250) NULL'
    elif 'date' in word: return '[DATE] NULL'
    else: return '--{Write var type}'

list_label_names= list(map(lambda x: clean_label(x[0]), sp_text))
list_labels= list(map(lambda x: 
                '['
                + x
                + '] '
                + is_nvarchar(x)
                , list_label_names))


In [206]:
#Table names

def clean_name(word):
    word= word.strip() \
        .replace('.', '].[')

    return word

list_table_names= list(map(lambda x: clean_name(x[1]), sp_text))

In [207]:
#Table names (alternative)
list_table_alt_names= list(map(lambda x: clean_label(x[1].split('.')[0]), sp_text))
list_column_alt_names= list(map(lambda x: clean_label(x[1].split('.')[1]), sp_text))

## Desire Results

In [208]:
# Create table sypnase
print(
"""IF OBJECT_ID('acuity.{0}') IS NOT NULL
    DROP  TABLE [acuity].[{0}];
GO

CREATE TABLE [acuity].[{0}]
( 
        {1}
)
WITH
(
    DISTRIBUTION = ROUND_ROBIN,
	HEAP
)
GO
""".format(table_name,
        '\n\t,'.join(list_labels)))

IF OBJECT_ID('acuity.s_program_oe_rate') IS NOT NULL
    DROP  TABLE [acuity].[s_program_oe_rate];
GO

CREATE TABLE [acuity].[s_program_oe_rate]
( 
        [enrollment_id] [NVARCHAR] (250) NULL
	,[student_id] [NVARCHAR] (250) NULL
	,[client_id] [NVARCHAR] (250) NULL
	,[program_oe_id] [NVARCHAR] (250) NULL
	,[next_enrollment_id] [NVARCHAR] (250) NULL
	,[next_program_oe_id] [NVARCHAR] (250) NULL
	,[previous_enrollment_id] [NVARCHAR] (250) NULL
	,[previous_program_oe_id] [NVARCHAR] (250) NULL
)
WITH
(
    DISTRIBUTION = ROUND_ROBIN,
	HEAP
)
GO



In [209]:
#Create Stored Procedure
list_query_var= list(map(lambda x,y: 
                x
                + ' AS '
                + y
                
                ,list_table_names,
                list_label_names))

print(
"""IF OBJECT_ID ( '[acuity].[sp_{0}]' ) IS NOT NULL   
    DROP PROCEDURE [acuity].[sp_{0}];  
GO  

CREATE PROC [acuity].[sp_{0}] AS BEGIN
    BEGIN TRANSACTION

        DECLARE @ErrorMessage NVARCHAR(4000);  
        DECLARE @ErrorSeverity INT;  
        DECLARE @ErrorState INT; 
        DECLARE @initial_time DATETIME;
        DECLARE @final_time DATETIME;
        DECLARE @time_diff_seconds FLOAT;
        DECLARE @table_name NVARCHAR(4000); 
        DECLARE @validate_query_result INT;

        BEGIN TRY  


            SET @initial_time  = CURRENT_TIMESTAMP;
		    SET @table_name = '{0}';

            
            DELETE FROM [acuity].[{0}]; 

            INSERT INTO [acuity].[{0}]

            ## Complete query
            SELECT 
                {1}

			
            SET @validate_query_result =  (SELECT count(*) FROM [acuity].[{0}]);

			IF @validate_query_result = 0
                BEGIN
                RAISERROR ('There are no rows in the source table', -- Message text.
                            16, -- Severity,
                            1 -- State,
                            );
                END ;

            SET @final_time  = CURRENT_TIMESTAMP;
            SET @time_diff_seconds = DATEDIFF(microsecond,@initial_time,@final_time) /1000000.0 ;

            INSERT INTO acuity.execution_history
                values(@table_name,@validate_query_result,'Table has been updated successfully',@time_diff_seconds , 'true', @final_time)


            COMMIT; 
            PRINT 'Table has been updated successfully'
        END TRY  
        BEGIN CATCH  
            ROLLBACK;
            
            SELECT   
                @ErrorMessage = ERROR_MESSAGE(),  
                @ErrorSeverity = ERROR_SEVERITY(),  
                @ErrorState = ERROR_STATE();  
        
            RAISERROR (@ErrorMessage, 
                    @ErrorSeverity, 
                    @ErrorState 
                    ); 

            SET @final_time  = CURRENT_TIMESTAMP;
            SET @time_diff_seconds = DATEDIFF(microsecond,@initial_time,@final_time) /1000000.0 ;
            
            INSERT INTO acuity.execution_history
            values(@table_name,@validate_query_result,@ErrorMessage,@time_diff_seconds , 'false', @final_time)

        END CATCH
END
GO
""".format(table_name,
        '\n\t\t\t\t,'.join(list_query_var)))

IF OBJECT_ID ( '[acuity].[sp_s_program_oe_rate]' ) IS NOT NULL   
    DROP PROCEDURE [acuity].[sp_s_program_oe_rate];  
GO  

CREATE PROC [acuity].[sp_s_program_oe_rate] AS BEGIN
    BEGIN TRANSACTION

        DECLARE @ErrorMessage NVARCHAR(4000);  
        DECLARE @ErrorSeverity INT;  
        DECLARE @ErrorState INT; 
        DECLARE @initial_time DATETIME;
        DECLARE @final_time DATETIME;
        DECLARE @time_diff_seconds FLOAT;
        DECLARE @table_name NVARCHAR(4000); 
        DECLARE @validate_query_result INT;

        BEGIN TRY  


            SET @initial_time  = CURRENT_TIMESTAMP;
		    SET @table_name = 's_program_oe_rate';

            
            DELETE FROM [acuity].[s_program_oe_rate]; 

            INSERT INTO [acuity].[s_program_oe_rate]

            ## Complete query
            SELECT 
                [S_staged_next_enrollment].[Enrollment Id] AS enrollment_id
				,[S_staged_next_enrollment].[Student Id] AS student_id
				,[S_staged_next_enrollment].[Client 

In [210]:
#Alternative query
list_query_var= list(map(lambda x,y,z:
                '['
                + x
                + '].['
                + y
                + '] AS '
                + z
                
                ,list_table_alt_names
                ,list_column_alt_names
                ,list_label_names))

print(
"""IF OBJECT_ID ( '[acuity].[sp_{0}]' ) IS NOT NULL   
    DROP PROCEDURE [acuity].[sp_{0}];  
GO  

CREATE PROC [acuity].[sp_{0}] AS BEGIN
    BEGIN TRANSACTION

        DECLARE @ErrorMessage NVARCHAR(4000);  
        DECLARE @ErrorSeverity INT;  
        DECLARE @ErrorState INT; 
        DECLARE @initial_time DATETIME;
        DECLARE @final_time DATETIME;
        DECLARE @time_diff_seconds FLOAT;
        DECLARE @table_name NVARCHAR(4000); 
        DECLARE @validate_query_result INT;

        BEGIN TRY  


            SET @initial_time  = CURRENT_TIMESTAMP;
		    SET @table_name = '{0}';

            
            DELETE FROM [acuity].[{0}]; 

            INSERT INTO [acuity].[{0}]

            ## Complete query
            SELECT 
                {1}

			
            SET @validate_query_result =  (SELECT count(*) FROM [acuity].[{0}]);

			IF @validate_query_result = 0
                BEGIN
                RAISERROR ('There are no rows in the source table', -- Message text.
                            16, -- Severity,
                            1 -- State,
                            );
                END ;

            SET @final_time  = CURRENT_TIMESTAMP;
            SET @time_diff_seconds = DATEDIFF(microsecond,@initial_time,@final_time) /1000000.0 ;

            INSERT INTO acuity.execution_history
                values(@table_name,@validate_query_result,'Table has been updated successfully',@time_diff_seconds , 'true', @final_time)


            COMMIT; 
            PRINT 'Table has been updated successfully'
        END TRY  
        BEGIN CATCH  
            ROLLBACK;
            
            SELECT   
                @ErrorMessage = ERROR_MESSAGE(),  
                @ErrorSeverity = ERROR_SEVERITY(),  
                @ErrorState = ERROR_STATE();  
        
            RAISERROR (@ErrorMessage, 
                    @ErrorSeverity, 
                    @ErrorState 
                    ); 

            SET @final_time  = CURRENT_TIMESTAMP;
            SET @time_diff_seconds = DATEDIFF(microsecond,@initial_time,@final_time) /1000000.0 ;
            
            INSERT INTO acuity.execution_history
            values(@table_name,@validate_query_result,@ErrorMessage,@time_diff_seconds , 'false', @final_time)

        END CATCH
END
GO
""".format(table_name,
        '\n\t\t\t\t,'.join(list_query_var)))

IF OBJECT_ID ( '[acuity].[sp_s_program_oe_rate]' ) IS NOT NULL   
    DROP PROCEDURE [acuity].[sp_s_program_oe_rate];  
GO  

CREATE PROC [acuity].[sp_s_program_oe_rate] AS BEGIN
    BEGIN TRANSACTION

        DECLARE @ErrorMessage NVARCHAR(4000);  
        DECLARE @ErrorSeverity INT;  
        DECLARE @ErrorState INT; 
        DECLARE @initial_time DATETIME;
        DECLARE @final_time DATETIME;
        DECLARE @time_diff_seconds FLOAT;
        DECLARE @table_name NVARCHAR(4000); 
        DECLARE @validate_query_result INT;

        BEGIN TRY  


            SET @initial_time  = CURRENT_TIMESTAMP;
		    SET @table_name = 's_program_oe_rate';

            
            DELETE FROM [acuity].[s_program_oe_rate]; 

            INSERT INTO [acuity].[s_program_oe_rate]

            ## Complete query
            SELECT 
                [s_staged_next_enrollment].[enrollment_id] AS enrollment_id
				,[s_staged_next_enrollment].[student_id] AS student_id
				,[s_staged_next_enrollment].[client_